# **Data Preprocessing**

In this notebook we aim to prepare the data for modelling.
Data preprocessing will take place in x stages:

* **1. Loading in the Data**
* **2. Merging the Dataframes**
* **3. Creating New Variables**
* **4. Filling in Missing Values**
* **5. Adding Dummy Variables**

After preprocessing is complete, the data will be ready for training a machine learning model 

## **Dependencies**

In [1]:
import pandas as pd
import numpy as np

## **1. Loading in the Data**

We are going to read in our six dataframes directly from Github, which will prevent other users from having to locally download the csv files

In [3]:
race_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/race.csv'
races = pd.read_csv(race_url)

results_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/results.csv'
results = pd.read_csv(results_url)

qualifying_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/qualifying_results.csv'
qualifying = pd.read_csv(qualifying_url)

driver_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/driver_standings.csv'
driver_standings = pd.read_csv(driver_url)

constructor_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/constructor_standings.csv'
constructor_standings = pd.read_csv(constructor_url)

weather_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/weather_info.csv'
weather = pd.read_csv(weather_url)